In [ ]:
import json
from bs4 import BeautifulSoup
import re
import numpy as np
import string
import torch
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install  -q datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q evaluate

!pip install -q git+https://github.com/huggingface/accelerate

!git clone https://github.com/huggingface/transformers.git
!cd transformers
!pip install -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metada

original data

In [ ]:
with open('/content/drive/MyDrive/extra_translations/makarem/makarem_train.json', 'r') as f:
  train_data_list = json.load(f)

with open('/content/drive/MyDrive/extra_translations/makarem/makarem_dev.json', 'r') as f:
  dev_data_list = json.load(f)

# Evaluate Original

In [ ]:
from datasets import load_dataset

# data_files = {"train": "/content/drive/MyDrive/train.json", "validation": "/content/drive/MyDrive/dev.json"}

train_path = "/content/drive/MyDrive/extra_translations/makarem/makarem_train.json"
valdiation_path = "/content/drive/MyDrive/extra_translations/makarem/makarem_dev.json"

data_files = {"train": train_path, "validation": valdiation_path}
dataset = load_dataset("json", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data = dataset['train']
dev_data = dataset['validation']

# Evaluate Back-translation

In [ ]:
# combine data from back-translation
from datasets import Dataset
import random

with open('/content/drive/MyDrive/back_translated_singleverse.json', 'r') as f:
  extra_data_list = json.load(f)


total_data = extra_data_list + train_data_list
random.shuffle(total_data)

train_data = Dataset.from_list(total_data)
dev_data = Dataset.from_list(dev_data_list)


# Evaluate extra context

In [ ]:
from datasets import Dataset
import random

with open('/content/drive/MyDrive/extra_translations.jsonl', 'r') as f:
  extra_data_list = []
  for line in f:
   data = json.loads(line)
   extra_data_list.append(data)


total_data = extra_data_list + train_data_list
random.shuffle(total_data)

train_data = Dataset.from_list(total_data)
dev_data = Dataset.from_list(dev_data_list)

# Evaluate Automatic Questions

In [ ]:
from datasets import Dataset
import random

with open('/content/drive/MyDrive/automatic.jsonl', 'r') as f:
  extra_data_list = []
  for line in f:
   data = json.loads(line)
   extra_data_list.append(data)


total_data = extra_data_list + train_data_list
random.shuffle(total_data)

train_data = Dataset.from_list(total_data)
dev_data = Dataset.from_list(dev_data_list)

# Evaluate all

In [ ]:
from datasets import Dataset
import random

with open('/content/drive/MyDrive/extra_translations/makarem/makarem_train.json', 'r') as f:
  original_data = json.load(f)

with open('/content/drive/MyDrive/extra_translations/makarem/makarem_dev.json', 'r') as f:
  dev_data_list = json.load(f)

with open('/content/drive/MyDrive/back_translated_singleverse.json', 'r') as f:
  back_translation = json.load(f)

with open('/content/drive/MyDrive/extra_translations.jsonl', 'r') as f:
  extra_data_list = []
  for line in f:
   data = json.loads(line)
   extra_data_list.append(data)

with open('/content/drive/MyDrive/automatic.jsonl', 'r') as f:
  automatic = []
  for line in f:
   data = json.loads(line)
   automatic.append(data)

total_data = original_data + back_translation + extra_data_list + automatic
random.shuffle(total_data)

train_data = Dataset.from_list(total_data)
dev_data = Dataset.from_list(dev_data_list)

# Model

In [ ]:
from transformers import AutoTokenizer

# model_checkpoint = "/content/drive/MyDrive/Quran_QA4/quranBERT_moreepochs/checkpoint-3118"
model_checkpoint = "/content/drive/MyDrive/albert_pquad/checkpoint-8000"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]

        start_char = answer[0]["start_char"]
        end_char = answer[0]["start_char"] + len(answer[0]['text'])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = train_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_data.column_names,
)
len(train_data), len(train_dataset)

Map:   0%|          | 0/10571 [00:00<?, ? examples/s]

(10571, 10576)

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["pq_id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = dev_data.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dev_data.column_names,
)
len(dev_data), len(validation_dataset)

Map:   0%|          | 0/392 [00:00<?, ? examples/s]

(392, 392)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
from tqdm.auto import tqdm
import collections

n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["pq_id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = []
    for ex in examples:
      new_ex = {}
      new_ex['id'] = ex["pq_id"]
      new_ex['answers'] = {}
      new_ex['answers']['answer_start'] = [ex['answers'][0]['start_char']]
      new_ex['answers']['text'] = [ex['answers'][0]['text']]
      theoretical_answers.append(new_ex)


    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "c",
    evaluation_strategy="no",
    save_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
)

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.631100


Step,Training Loss
500,0.631100
1000,0.400400
1500,0.274200
2000,0.189600
2500,0.192400
3000,0.114400
3500,0.090200
4000,0.090500
4500,0.049800
5000,0.057900


TrainOutput(global_step=13220, training_loss=0.09329360128350771, metrics={'train_runtime': 3394.2397, 'train_samples_per_second': 31.159, 'train_steps_per_second': 3.895, 'total_flos': 1751687264747520.0, 'train_loss': 0.09329360128350771, 'epoch': 10.0})

In [ ]:
import evaluate

metric = evaluate.load("squad")
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, dev_data)

  0%|          | 0/392 [00:00<?, ?it/s]

{'exact_match': 83.41836734693878, 'f1': 87.34797475740565}